In [3]:
import numpy as np
import pandas as pd

# Simple competing risks example

In [28]:
N = 10
data = pd.DataFrame({"id": np.arange(N), "sex":np.random.binomial(n=1,p=0.5,size=N)})
data['source'] = 0
data['target'] = np.random.choice(a=[1,2], size=N, p=[0.5,0.5])
data['transition'] = data['target']
data['T'] = np.round(np.random.exponential(scale=(1+data['sex'].values))) + 1


data

,id,sex,source,target,transition,T
0,0,1,0,1,1,1.0
1,1,0,0,1,1,3.0
2,2,0,0,2,2,2.0
3,3,1,0,1,1,1.0
4,4,0,0,1,1,2.0
5,5,1,0,2,2,5.0
6,6,1,0,1,1,4.0
7,7,1,0,1,1,2.0
8,8,0,0,2,2,2.0
9,9,1,0,2,2,6.0
